In [1]:
#imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

In [2]:
all_matches = []
main_url= "https://fbref.com/en/comps/106/Womens-World-Cup-Stats"

In [3]:
data = requests.get(main_url) #html scraped
soup = BeautifulSoup(data.text) #parsing
standings_table = soup.select('table.stats_table')[8] #css selector of the main table

links = [l.get("href") for l in standings_table.find_all('a')] # get the links in the table
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/97d1aa04/2023/Spain-Women-Stats',
 'https://fbref.com/en/squads/5849ebe1/2023/England-Women-Stats',
 'https://fbref.com/en/squads/4173add7/2023/Sweden-Women-Stats',
 'https://fbref.com/en/squads/a3aba642/2023/Australia-Women-Stats',
 'https://fbref.com/en/squads/4210f192/2023/Japan-Women-Stats',
 'https://fbref.com/en/squads/064c6283/2023/France-Women-Stats',
 'https://fbref.com/en/squads/ec991a3d/2023/Netherlands-Women-Stats',
 'https://fbref.com/en/squads/1fe6955e/2023/Colombia-Women-Stats',
 'https://fbref.com/en/squads/1c912aa0/2023/United-States-Women-Stats',
 'https://fbref.com/en/squads/2bbd2880/2023/Nigeria-Women-Stats',
 'https://fbref.com/en/squads/a8855f01/2023/Denmark-Women-Stats',
 'https://fbref.com/en/squads/63be3c0c/2023/Morocco-Women-Stats',
 'https://fbref.com/en/squads/341db2fd/2023/Jamaica-Women-Stats',
 'https://fbref.com/en/squads/b08f950e/2023/Switzerland-Women-Stats',
 'https://fbref.com/en/squads/1ec54f37/2023/Norway-Women-Stats',


In [4]:

for team_url in team_urls: # to get each team in the main table
    time.sleep(10)
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    html= StringIO(data.text)
    try:
        matches = pd.read_html(html, match="Scores & Fixtures")[0]
    except ValueError:
        continue
    time.sleep(3)
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and '/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    html= StringIO(data.text)
    time.sleep(3)
    try:
        shooting = pd.read_html(html, match="Shooting")[0]
    except ValueError:
        continue
    shooting.columns = shooting.columns.droplevel()
    time.sleep(3)
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "World Cup"]

    team_data["Season"] = 2023
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(3)

In [5]:
matches_2023 = pd.concat(all_matches)

In [6]:
matches_2023.to_csv("WWC_2023_matches.csv")

In [7]:
matches_2023 = pd.read_csv("WWC_2023_matches.csv")
matches_2023

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,SoT,SoT%,G/Sh,G/SoT,Dist,FK,PK,PKatt,Season,Team
0,12,2023-07-21,19:30,World Cup,Group stage,Fri,Neutral,W,3,0,...,11.0,24.4,0.04,0.18,14.7,0.0,0,1,2023,Spain Women
1,13,2023-07-26,19:30,World Cup,Group stage,Wed,Neutral,W,5,0,...,13.0,59.1,0.23,0.38,15.6,0.0,0,0,2023,Spain Women
2,14,2023-07-31,19:00,World Cup,Group stage,Mon,Neutral,L,0,4,...,2.0,20.0,0.00,0.00,22.1,0.0,0,0,2023,Spain Women
3,15,2023-08-05,17:00,World Cup,Round of 16,Sat,Neutral,W,5,1,...,10.0,38.5,0.19,0.50,16.0,1.0,0,0,2023,Spain Women
4,16,2023-08-11,13:00,World Cup,Quarter-finals,Fri,Neutral,W,2,1,...,7.0,25.9,0.04,0.14,18.6,1.0,1,1,2023,Spain Women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,5,2023-07-29,20:30,World Cup,Group stage,Sat,Neutral,L,0,1,...,4.0,30.8,0.00,0.00,23.0,0.0,0,0,2023,Panama Women
124,6,2023-08-02,20:00,World Cup,Group stage,Wed,Neutral,L,3,6,...,2.0,40.0,0.40,1.00,24.6,2.0,1,1,2023,Panama Women
125,2,2023-07-22,13:00,World Cup,Group stage,Sat,Neutral,L,0,3,...,0.0,NaN,NaN,NaN,NaN,0.0,0,0,2023,Vietnam Women
126,3,2023-07-27,19:30,World Cup,Group stage,Thu,Neutral,L,0,2,...,1.0,20.0,0.00,0.00,26.4,0.0,0,0,2023,Vietnam Women


In [8]:
matches_2023 = matches_2023.drop(columns=['Unnamed: 0'])
matches_2023

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,SoT,SoT%,G/Sh,G/SoT,Dist,FK,PK,PKatt,Season,Team
0,2023-07-21,19:30,World Cup,Group stage,Fri,Neutral,W,3,0,cr Costa Rica,...,11.0,24.4,0.04,0.18,14.7,0.0,0,1,2023,Spain Women
1,2023-07-26,19:30,World Cup,Group stage,Wed,Neutral,W,5,0,zm Zambia,...,13.0,59.1,0.23,0.38,15.6,0.0,0,0,2023,Spain Women
2,2023-07-31,19:00,World Cup,Group stage,Mon,Neutral,L,0,4,jp Japan,...,2.0,20.0,0.00,0.00,22.1,0.0,0,0,2023,Spain Women
3,2023-08-05,17:00,World Cup,Round of 16,Sat,Neutral,W,5,1,ch Switzerland,...,10.0,38.5,0.19,0.50,16.0,1.0,0,0,2023,Spain Women
4,2023-08-11,13:00,World Cup,Quarter-finals,Fri,Neutral,W,2,1,nl Netherlands,...,7.0,25.9,0.04,0.14,18.6,1.0,1,1,2023,Spain Women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2023-07-29,20:30,World Cup,Group stage,Sat,Neutral,L,0,1,jm Jamaica,...,4.0,30.8,0.00,0.00,23.0,0.0,0,0,2023,Panama Women
124,2023-08-02,20:00,World Cup,Group stage,Wed,Neutral,L,3,6,fr France,...,2.0,40.0,0.40,1.00,24.6,2.0,1,1,2023,Panama Women
125,2023-07-22,13:00,World Cup,Group stage,Sat,Neutral,L,0,3,us USA,...,0.0,NaN,NaN,NaN,NaN,0.0,0,0,2023,Vietnam Women
126,2023-07-27,19:30,World Cup,Group stage,Thu,Neutral,L,0,2,pt Portugal,...,1.0,20.0,0.00,0.00,26.4,0.0,0,0,2023,Vietnam Women
